# Modules

In [1]:
import tensorflow as tf
import scipy
from tensorflow.python.client import device_lib
from tensorflow.keras.applications import InceptionV3, ResNet152V2, VGG19
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D, BatchNormalization,AveragePooling2D, TimeDistributed, LSTM
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow as tf
print(tf.__version__)

2.10.1


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15269815223788097233
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6284115968
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7758084166447153520
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

# Dataset preparation

In [5]:
batchsize = 8

In [6]:
# Training dataset

train_data_gen = ImageDataGenerator(rotation_range=0.2, 
                                        shear_range=0.2,
                                        zoom_range=0.2, 
                                        width_shift_range=0.2,
                                        height_shift_range=0.2, 
                                        rescale=1./255,
                                        validation_split=0.2)

train_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='training'
                                                    )

validation_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='validation'
                                                    )

Found 64719 images belonging to 2 classes.
Found 16179 images belonging to 2 classes.


In [7]:
# Testing dataset

test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = test_data_gen.flow_from_directory('dataset/test', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical'
                                                    )

Found 4000 images belonging to 2 classes.


# Model preparation

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [69]:
# Base Model : InceptionV3

# Get knowledge from InceptionV3 pretrained model, so we don't need to train the model from scratch
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)

# The only layers we need to train
# This is the layers' illustration https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fthe-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480&psig=AOvVaw0CYHQ2oruhWMmKK2Lxm5cN&ust=1668996857968000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCLDJs9TYu_sCFQAAAAAdAAAAABAU
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(64, activation = "relu")(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False

checkpoint = ModelCheckpoint('models/InceptionV3', 
                             monitor='val_loss', 
                             save_best_only=True, 
                             verbose=3)
    
print('done')

done


In [13]:
#Base Model : ResNet152V2

# Get knowledge from ResNet152V2 pretrained model, so we don't need to train the model from scratch
bmodel = ResNet152V2(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel = bmodel.output
hmodel = AveragePooling2D(pool_size = (3,3))(hmodel)
hmodel = Flatten(name= 'flatten')(hmodel)

# The only layers we need to train
# This is the layers' illustration https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fthe-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480&psig=AOvVaw0CYHQ2oruhWMmKK2Lxm5cN&ust=1668996857968000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCLDJs9TYu_sCFQAAAAAdAAAAABAU

hmodel = Dense(256, activation = "relu")(hmodel)
hmodel = Dropout(0.25)(hmodel)
hmodel = Dense(256, activation = "relu")(hmodel)
hmodel = Dropout(0.25)(hmodel)
hmodel = Dense(256, activation = "relu")(hmodel)
hmodel = Dropout(0.25)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)


for layer in bmodel.layers:
    layer.trainable = False

checkpoint = ModelCheckpoint('models/ResNet152V2', 
                             monitor='val_loss', 
                             save_best_only=True, 
                             verbose=3)

print('done')

done


In [48]:
# Base Model : CNN LSTM
# Did some modifications according to https://www.tensorflow.org/versions/r2.2/api_docs/python/tf/keras/layers/LSTM

bmodelVGG = VGG19(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel1 = Sequential()
hmodel1 = bmodelVGG.output
hmodel1 = AveragePooling2D(pool_size =(3,3), padding='same')(hmodel1)
hmodel1 = TimeDistributed(Flatten())(hmodel1)

#Train this layer 
hmodel1 = LSTM(64, dropout=0.5,  recurrent_dropout=0, unroll=False, activation='tanh')(hmodel1)
hmodel1 = BatchNormalization()(hmodel1)
hmodel1 = Dropout(0.25)(hmodel1)
hmodel1 = Dense(32, activation = "relu", input_shape=())(hmodel1)
hmodel1 = Dropout(0.25)(hmodel1)
hmodel1 = Dense(16, activation = "relu")(hmodel1)
hmodel1 = Dropout(0.25)(hmodel1)
hmodel1 = Dense(8, activation = "relu")(hmodel1)
hmodel1 = Dropout(0.25)(hmodel1)
hmodel1 = Dense(1, activation='sigmoid')(hmodel1)
model = Model(inputs=bmodelVGG.input, outputs=hmodel1)

for layer in bmodelVGG.layers:
    layer.trainable = False

checkpoint = ModelCheckpoint('models/CNN_LSTM', 
                             monitor='val_loss', 
                             save_best_only=True, 
                             verbose=3)

print('done')

done


In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 86, 86, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 40, 40, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 42, 42, 64)   0           ['conv1_conv[0][0]']             
                                                                                              

                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 20, 20, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 20, 20, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 20, 20, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_blo

 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 10, 10, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 10, 10, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_out (Add)         (None, 10, 10, 512)  0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_conv[0][0]']    
                                                                                                  
 conv3_blo

                                                                                                  
 conv3_block5_1_relu (Activatio  (None, 10, 10, 128)  0          ['conv3_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block5_2_pad (ZeroPaddin  (None, 12, 12, 128)  0          ['conv3_block5_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 10, 10, 128)  147456      ['conv3_block5_2_pad[0][0]']     
                                                                                                  
 conv3_block5_2_bn (BatchNormal  (None, 10, 10, 128)  512        ['conv3_block5_2_conv[0][0]']    
 ization) 

 ormalization)                                                                                    
                                                                                                  
 conv3_block8_preact_relu (Acti  (None, 10, 10, 512)  0          ['conv3_block8_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv3_block8_1_conv (Conv2D)   (None, 10, 10, 128)  65536       ['conv3_block8_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv3_block8_1_bn (BatchNormal  (None, 10, 10, 128)  512        ['conv3_block8_1_conv[0][0]']    
 ization)                                                                                         
          

...

 lization)                                                                                        
                                                                                                  
 conv4_block29_1_relu (Activati  (None, 5, 5, 256)   0           ['conv4_block29_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block29_2_pad (ZeroPaddi  (None, 7, 7, 256)   0           ['conv4_block29_1_relu[0][0]']   
 ng2D)                                                                                            
                                                                                                  
 conv4_block29_2_conv (Conv2D)  (None, 5, 5, 256)    589824      ['conv4_block29_2_pad[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block32_preact_bn (Batch  (None, 5, 5, 1024)  4096        ['conv4_block31_out[0][0]']      
 Normalization)                                                                                   
                                                                                                  
 conv4_block32_preact_relu (Act  (None, 5, 5, 1024)  0           ['conv4_block32_preact_bn[0][0]']
 ivation)                                                                                         
                                                                                                  
 conv4_block32_1_conv (Conv2D)  (None, 5, 5, 256)    262144      ['conv4_block32_preact_relu[0][0]
                                                                 ']                               
                                                                                                  
 conv4_block32_1_bn (BatchNorma  (None, 5, 5, 256)   1024        ['conv4_block32_1_conv[0][0]']   
 lization)

 lization)                                                                                        
                                                                                                  
 conv4_block34_2_relu (Activati  (None, 5, 5, 256)   0           ['conv4_block34_2_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block34_3_conv (Conv2D)  (None, 5, 5, 1024)   263168      ['conv4_block34_2_relu[0][0]']   
                                                                                                  
 conv4_block34_out (Add)        (None, 5, 5, 1024)   0           ['conv4_block33_out[0][0]',      
                                                                  'conv4_block34_3_conv[0][0]']   
                                                                                                  
 conv4_blo

 conv5_block1_1_bn (BatchNormal  (None, 3, 3, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 3, 3, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_2_pad (ZeroPaddin  (None, 5, 5, 512)   0           ['conv5_block1_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv5_block1_2_conv (Conv2D)   (None, 3, 3, 512)    2359296     ['conv5_block1_2_pad[0][0]']     
          

                                                                                                  
 conv5_block3_out (Add)         (None, 3, 3, 2048)   0           ['conv5_block2_out[0][0]',       
                                                                  'conv5_block3_3_conv[0][0]']    
                                                                                                  
 post_bn (BatchNormalization)   (None, 3, 3, 2048)   8192        ['conv5_block3_out[0][0]']       
                                                                                                  
 post_relu (Activation)         (None, 3, 3, 2048)   0           ['post_bn[0][0]']                
                                                                                                  
 average_pooling2d (AveragePool  (None, 1, 1, 2048)  0           ['post_relu[0][0]']              
 ing2D)                                                                                           
          

# Train the model

In [16]:
# patience = max num of epochs when accuracy is not increasing
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks = [checkpoint, earlystop, learning_rate]

In [18]:
# Resnet152V2
opt = Adam(lr=1e-4, decay=1e-4 / 50)
try:
    model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data,
          steps_per_epoch=train_data.samples//batchsize,
          validation_data=validation_data,
          validation_steps=validation_data.samples//batchsize,
          callbacks=callbacks,
          epochs=50)
    
    model.save('Resnet152V2_Drowsiness.h5')
except Exception as e:
    print(e)

Epoch 1/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.2311 - accuracy: 0.9048
Epoch 1: val_loss improved from inf to 0.25456, saving model to models\ResNet152V2


INFO:tensorflow:Assets written to: models\ResNet152V2\assets


INFO:tensorflow:Assets written to: models\ResNet152V2\assets


8089/8089 [==============================] - 384s 46ms/step - loss: 0.2310 - accuracy: 0.9048 - val_loss: 0.2546 - val_accuracy: 0.8907 - lr: 1.0000e-04
Epoch 2/50
8089/8089 [==============================] - ETA: 0s - loss: 0.1640 - accuracy: 0.9345
Epoch 2: val_loss did not improve from 0.25456
8089/8089 [==============================] - 320s 39ms/step - loss: 0.1640 - accuracy: 0.9345 - val_loss: 0.2832 - val_accuracy: 0.8725 - lr: 1.0000e-04
Epoch 3/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.1470 - accuracy: 0.9415
Epoch 3: val_loss improved from 0.25456 to 0.23575, saving model to models\ResNet152V2


INFO:tensorflow:Assets written to: models\ResNet152V2\assets


INFO:tensorflow:Assets written to: models\ResNet152V2\assets


8089/8089 [==============================] - 360s 45ms/step - loss: 0.1470 - accuracy: 0.9416 - val_loss: 0.2357 - val_accuracy: 0.8994 - lr: 1.0000e-04
Epoch 4/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.1374 - accuracy: 0.9468
Epoch 4: val_loss did not improve from 0.23575
8089/8089 [==============================] - 319s 39ms/step - loss: 0.1374 - accuracy: 0.9468 - val_loss: 0.2725 - val_accuracy: 0.8850 - lr: 1.0000e-04
Epoch 5/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.1338 - accuracy: 0.9484
Epoch 5: val_loss did not improve from 0.23575
8089/8089 [==============================] - 320s 40ms/step - loss: 0.1338 - accuracy: 0.9484 - val_loss: 0.2799 - val_accuracy: 0.8835 - lr: 1.0000e-04
Epoch 6/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.1255 - accuracy: 0.9519
Epoch 6: val_loss did not improve from 0.23575

Epoch 6: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
8089/8089 [==================

In [71]:
# InceptionV3

try:
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data,
          steps_per_epoch=train_data.samples//batchsize,
          validation_data=validation_data,
          validation_steps=validation_data.samples//batchsize,
          callbacks=callbacks,
          epochs=50)
    
    model.save('InceptionV3_Drowsiness.h5')
except Exception as e:
    print(e)

Epoch 1/50
8089/8089 [==============================] - ETA: 0s - loss: 0.3256 - accuracy: 0.8822
Epoch 1: val_loss improved from inf to 0.49945, saving model to models\InceptionV3


INFO:tensorflow:Assets written to: models\InceptionV3\assets


INFO:tensorflow:Assets written to: models\InceptionV3\assets


8089/8089 [==============================] - 443s 54ms/step - loss: 0.3256 - accuracy: 0.8822 - val_loss: 0.4995 - val_accuracy: 0.8157 - lr: 0.0010
Epoch 2/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.2848 - accuracy: 0.9072
Epoch 2: val_loss did not improve from 0.49945
8089/8089 [==============================] - 498s 62ms/step - loss: 0.2848 - accuracy: 0.9072 - val_loss: 0.5343 - val_accuracy: 0.8030 - lr: 0.0010
Epoch 3/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.2743 - accuracy: 0.9097
Epoch 3: val_loss improved from 0.49945 to 0.36467, saving model to models\InceptionV3


INFO:tensorflow:Assets written to: models\InceptionV3\assets


...

INFO:tensorflow:Assets written to: models\InceptionV3\assets


8089/8089 [==============================] - 629s 78ms/step - loss: 0.2645 - accuracy: 0.9275 - val_loss: 0.3120 - val_accuracy: 0.9162 - lr: 1.0000e-04
Epoch 18/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2662 - accuracy: 0.9274
Epoch 18: val_loss did not improve from 0.31196
8089/8089 [==============================] - 559s 69ms/step - loss: 0.2662 - accuracy: 0.9274 - val_loss: 0.3212 - val_accuracy: 0.9104 - lr: 1.0000e-04
Epoch 19/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2677 - accuracy: 0.9286
Epoch 19: val_loss did not improve from 0.31196
8089/8089 [==============================] - 565s 70ms/step - loss: 0.2677 - accuracy: 0.9286 - val_loss: 0.3142 - val_accuracy: 0.9065 - lr: 1.0000e-04
Epoch 20/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2528 - accuracy: 0.9279
Epoch 20: val_loss improved from 0.31196 to 0.29997, saving model to models\InceptionV3


INFO:tensorflow:Assets written to: models\InceptionV3\assets


INFO:tensorflow:Assets written to: models\InceptionV3\assets


8089/8089 [==============================] - 605s 75ms/step - loss: 0.2528 - accuracy: 0.9279 - val_loss: 0.3000 - val_accuracy: 0.9165 - lr: 1.0000e-04
Epoch 21/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2681 - accuracy: 0.9300
Epoch 21: val_loss did not improve from 0.29997
8089/8089 [==============================] - 576s 71ms/step - loss: 0.2681 - accuracy: 0.9300 - val_loss: 0.3210 - val_accuracy: 0.9094 - lr: 1.0000e-04
Epoch 22/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2839 - accuracy: 0.9290
Epoch 22: val_loss did not improve from 0.29997
8089/8089 [==============================] - 558s 69ms/step - loss: 0.2839 - accuracy: 0.9290 - val_loss: 0.3109 - val_accuracy: 0.9167 - lr: 1.0000e-04
Epoch 23/50
8089/8089 [==============================] - ETA: 0s - loss: 0.3270 - accuracy: 0.9303
Epoch 23: val_loss did not improve from 0.29997

Epoch 23: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
8089/8089 [==========

In [47]:
# # CNN-LSTM

# try:
#     model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

#     model.fit(train_data,
#           steps_per_epoch=train_data.samples//batchsize,
#           validation_data=validation_data,
#           validation_steps=validation_data.samples//batchsize,
#           callbacks=callbacks,
#           epochs=50)
    
#     model.save('CNN_LSTM_Drowsiness.h5')
# except Exception as e:
#     print(e)

Epoch 1/50
Graph execution error:

Detected at node 'gradient_tape/model_10/dense_43/MatMul/MatMul' defined at (most recent call last):
    File "C:\Users\felix\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\felix\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\felix\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\felix\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\felix\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\felix\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\felix\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._r

# Model evaluation InceptionV3

In [72]:
acc_training, loss_training = model.evaluate(train_data)
print(acc_training)
print(loss_training)

8090/8090 [==============================] - 412s 51ms/step - loss: 0.2751 - accuracy: 0.9390
0.2751415967941284
0.9389514923095703


In [73]:
acc_val, loss_val = model.evaluate(validation_data)
print(acc_val)
print(loss_val)

2023/2023 [==============================] - 104s 51ms/step - loss: 0.3058 - accuracy: 0.9130
0.30575045943260193
0.9130353927612305


In [74]:
from tensorflow.keras.models import load_model
model = load_model('models/InceptionV3//')
acc_test, loss_test = model.evaluate(test_data)
print(acc_test)
print(loss_test)

500/500 [==============================] - 29s 55ms/step - loss: 0.5321 - accuracy: 0.8390
0.5320631265640259
0.8389999866485596


# Model evaluation Resnet 152V2

In [22]:
acc_training, loss_training = model.evaluate(train_data)
print(acc_training)
print(loss_training)

8090/8090 [==============================] - 267s 33ms/step - loss: 0.1278 - accuracy: 0.9490
0.12783199548721313
0.9490103125572205


In [23]:
acc_val, loss_val = model.evaluate(validation_data)
print(acc_val)
print(loss_val)

2023/2023 [==============================] - 65s 32ms/step - loss: 0.2431 - accuracy: 0.8941
0.24305808544158936
0.8940601944923401


In [27]:
from tensorflow.keras.models import load_model
model1 = load_model('models/ResNet152V2//')
acc_test, loss_test = model1.evaluate(test_data)
print(acc_test)
print(loss_test)

500/500 [==============================] - 20s 34ms/step - loss: 0.2278 - accuracy: 0.9060
0.22775539755821228
0.906000018119812
